### PreProcessing - Append Indicators



In [2]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy as np
import os
import urllib
import json
import requests
import matplotlib.pyplot as plt
from io import StringIO

from pathlib import Path
from eod import EodHistoricalData

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)

In [3]:
#paths for data - set prefix to location of Data folder
path_prefix = r'C:\Users\OEM\GDrive\WQU'
path_att = r'\Data\fundamentals_by_attribute'
path_fun = r'\Data\fundamentals_by_ticker'
path_std = r'\Data\standardised_fundamentals'
path_eda = r'\Data\exploratory_data_analysis'
path_rob = r'\Data\robust_scaling'
path_ind = r'\Data\indicator_prices'

In [4]:
#Instantiate datasource client
api_key = "618f834b7bfe27.18132752"
client = EodHistoricalData(api_key)

In [5]:
#get ticker symbols from exchange
resp = client.get_exchange_symbols(exchange='NYSE')

In [12]:
#create list of stock tickers
tickers = []
for i in range(len(resp)):
    if resp[i]['Type'] == 'Common Stock' or resp[i]['Type'] == 'Preferred Stock' :
        ticker = resp[i]['Code']
        tickers.append(ticker)

In [13]:
idx = tickers.index('UHAL-B')
tickers = tickers[:idx]
idx

2830

In [ ]:
tickers

In [ ]:
#calculate the return realised for a given percentage drawdown
for ticker in tickers:
    
    filepath = Path(path_prefix + path_rob + "/" + 'df_rob_{}.csv'.format(ticker))
    print(ticker)
    
    #if dataset for ticker exists load historic prices from api and calculate return
    if os.path.isfile(filepath) == True:
        prices = client.get_prices_eod(ticker)
        df_prices = pd.DataFrame(prices)
        df_prices.set_index('date', inplace=True)
        df_prices.index = pd.to_datetime(df_prices.index)
        df_prices['pct_ret'] = df_prices['adjusted_close'].pct_change()
        df_prices['log_ret'] = np.log(df_prices['adjusted_close']) - np.log(df_prices['adjusted_close'].shift(1))
        #df_prices['prev_close'] = np.nan
      
    for row in df_prices.index:
        row_price = df_prices.loc[row, 'adjusted_close']
        max_price = row_price
        drawdown = 0
        count = 0
        loc_row = df_prices.index.get_loc(row)
        loc_ac = df_prices.columns.get_loc('adjusted_close')
    
        while drawdown > -0.1 and (loc_row + count) <  df_prices.shape[0]:
     
                 iter_price = df_prices.iloc[loc_row + count, loc_ac]
                 if iter_price > max_price:
                     max_price = iter_price
                 drawdown = (iter_price - max_price) / max_price
                 count += 1
        
        ret = (iter_price - row_price) / row_price
        df_prices.loc[row, '10%DDret'] = ret
        
    df_prices.to_csv(path_prefix + path_ind + '\df_ind_{}.csv'.format(ticker))

In [15]:
#submission dates for financial statements are typically last day of the month which may be a weekend with no trading - so gaps need to be filled from trading days beforehand

for ticker in tickers:
    filepath = Path(path_prefix + path_rob + "/" + 'df_rob_{}.csv'.format(ticker))
    if os.path.isfile(filepath) == True:
        df_ind = pd.read_csv(path_prefix + path_ind + "/df_ind_{}.csv".format(ticker), index_col=0)
        df_ind.index = pd.to_datetime(df_ind.index)
        
        #read robust scaled fundamentals 
        df_rob = pd.read_csv(filepath, index_col=0)
        df_rob.index = pd.to_datetime(df_rob.index)
        print(ticker)
        #if adj close already appended from previous runs - remove
        if '10%DDret' in df_rob.columns :
            df_rob.drop(['10%DDret'], axis=1, inplace=True)
            
        elif 'DDret10%' in df_rob.columns :
            df_rob.drop(['DDret10%'], axis=1, inplace=True)
        
        #join close indicators to fundamentals on date 
        df_rob = df_rob.join(df_ind['10%DDret'], how='left')
        
        #for each row in fundamentals if no price was joined use the price from 1,2,3 or 4 days previous (accounting for non-trading over public holidays)
        for i in range(len(df_rob)):
            loc_ac = df_rob.columns.get_loc('10%DDret')
    
            if math.isnan(df_rob.iloc[i, loc_ac]):
                idx = df_rob.index[i]
                try:
                    idx = idx - dt.timedelta(days=1)
                    df_rob.iloc[i, loc_ac] = df_ind.loc[idx, '10%DDret']
                except:
                    print('Error on {}'.format(idx))

                try:
                    idx = idx - dt.timedelta(days=1)
                    df_rob.iloc[i, loc_ac] = df_ind.loc[idx, '10%DDret']
                except:
                    print('Error on {}'.format(idx))

                try:
                    idx = idx - dt.timedelta(days=1)
                    df_rob.iloc[i, loc_ac] = df_ind.loc[idx, '10%DDret']
                except:
                    print('Error on {}'.format(idx)) 

                try:
                    idx = idx - dt.timedelta(days=1)
                    df_rob.iloc[i, loc_ac] = df_ind.loc[idx, '10%DDret']
                except:
                    print('No price found for {}'.format(idx)) 
                    
    df_rob.to_csv(filepath)
    print(ticker + " completed")

A
Error on 2023-04-29 00:00:00
Error on 2022-07-30 00:00:00
Error on 2021-10-30 00:00:00
Error on 2021-01-30 00:00:00
Error on 2017-04-29 00:00:00
Error on 2016-07-30 00:00:00
Error on 2016-01-30 00:00:00
Error on 2011-07-30 00:00:00
Error on 2010-10-30 00:00:00
Error on 2010-01-30 00:00:00
Error on 2006-04-29 00:00:00
Error on 2005-07-30 00:00:00
Error on 2004-10-30 00:00:00
Error on 2000-04-29 00:00:00
A completed
AA
Error on 2019-06-29 00:00:00
Error on 2019-03-30 00:00:00
Error on 2018-09-29 00:00:00
Error on 2018-03-30 00:00:00
Error on 2017-12-30 00:00:00
AA completed
AACT
AACT completed
AAIC
Error on 2019-06-29 00:00:00
Error on 2019-03-30 00:00:00
Error on 2018-09-29 00:00:00
Error on 2018-03-30 00:00:00
Error on 2017-12-30 00:00:00
Error on 2013-06-29 00:00:00
Error on 2013-03-30 00:00:00
Error on 2013-03-29 00:00:00
Error on 2012-09-29 00:00:00
Error on 2007-09-29 00:00:00
Error on 2006-12-30 00:00:00
Error on 2002-06-29 00:00:00
Error on 2002-03-30 00:00:00
Error on 2002-03-

In [ ]:
'''
#submission dates for financial statements are typically last day of the month which may be a weekend with no trading - so gaps need to be filled from prices beforehand
#for each ticker get prices and add NaN column
for ticker in tickers:
    filepath = Path(path_prefix + path_rob + "/" + 'df_rob_{}.csv'.format(ticker))
    print(ticker)

    #if dataset for ticker exists load historic prices from api and calculate return
    if os.path.isfile(filepath) == True:
        prices = client.get_prices_eod(ticker)
        df_prices = pd.DataFrame(prices)
        df_prices.set_index('date', inplace=True)
        df_prices.index = pd.to_datetime(df_prices.index)
        df_prices['pct_ret'] = df_prices['adjusted_close'].pct_change()
        df_prices['log_ret'] = np.log(df_prices['adjusted_close']) - np.log(df_prices['adjusted_close'].shift(1))
        #df_prices['prev_close'] = np.nan

        marker = 0
        loc_row = 0
        
        for row in df_prices.index:
            #iterate through prices until drawdown detected
            #mark start of drawdown
            loc_row = df_prices.index.get_loc(row) + marker
            loc_ac = df_prices.columns.get_loc('adjusted_close')
            detect_start = df_prices.iloc[loc_row, loc_ac]
            detect_end = df_prices.iloc[loc_row + 1, loc_ac]
        
            #measuremeasure of drawdown once detected
            if detect_start > detect_end:
                count = 0
                measure_start = 0 
                measure_end = 0
                drawdown = 0
                
                while measure_start >= measure_end and drawdown > -0.1:
                    measure_start = df_prices.iloc[loc_row + count, loc_ac]
                    measure_end = df_prices.iloc[loc_row + count + 1, loc_ac]
                    count += 1
                    drawdown = (measure_start - detect_start) / detect_start
                
                if drawdown < -0.1:
                   entry = df_prices.loc[row, 'adjusted_close']
                   exit_ = measure_start
                   df_prices.loc[row, 'realised_ret'] = (exit_ - entry) / entry
                   
                elif measure_start >= measure_end:
                    marker = marker + count
                    
                else:
                    print('Error')
            
        



        for row in df_prices.index:
            buy_price = df_prices.loc[row, 'adjusted_close']
            drawdown = 0
            count = 1
            loc_ac = df_prices.columns.get_loc('adjusted_close')
            loc_row = df_prices.index.get_loc(row)

            dd_start = 
            
            while drawdown < 0.10 and (loc_row + count) < df_prices.shape[0]:
                running_price = df_prices.iloc[loc_row + count, loc_ac]

                
                drawdown = -(running_price - buy_price) / buy_price
                #print('row', row)
                #print('count', count)
                count += 1
                #print('drawdown', drawdown)
                #print('running_price', running_price)
                #print('buy_price', buy_price)
                #MDD = (Trough Value — Peak Value) / Peak Value
            pct_ret = (running_price - buy_price) / buy_price 
            df_prices.loc[row, 'DDret10%'] = pct_ret
            df_prices.loc[row, 'days_streak'] = count 
        
        df_prices.to_csv(path_prefix + path_ind + '\df_ind_{}'.format(ticker))
            
        #read standardised fundamentals (ensure index is datetime)
        df_std = pd.read_csv(filepath, index_col=0)
        df_std.index = pd.to_datetime(df_std.index)
        
        #if drawdown return already appended - remove
        if 'DDret10%' in df_std.columns:
            df_std.drop('DDret10%', inplace=True, axis=1)
        
        #join close prices to fundamentals on date 
        df_std = df_std.join(df_prices['DDret10%'], how='left')
        
        #for each row in fundamentals if no price was joined use the price from 1,2 or 3 days previous
        for i in range(len(df_std)):
            loc_dd = df_std.columns.get_loc('DDret10%')
    
            if math.isnan(df_std.iloc[i, loc_dd]):
                idx = df_std.index[i]
                try:
                    idx = idx - dt.timedelta(days=1)
                    df_std.iloc[i, loc_dd] = df_prices.loc[idx, 'DDret10%']
                except:
                    print('No price for {}'.format(idx))

                try:
                    idx = idx - dt.timedelta(days=1)
                    df_std.iloc[i, loc_dd] = df_prices.loc[idx, 'DDret10%']
                except:
                    print('No price for {}'.format(idx))

                try:
                    idx = idx - dt.timedelta(days=1)
                    df_std.iloc[i, loc_dd] = df_prices.loc[idx, 'DDret10%']
                except:
                    print('No price for {}'.format(idx)) 
                    
    df_std.to_csv(filepath)'''